In [ ]:
import os
import math
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import lib.datasets.synthetic as synthetic
import torchvision
import lib.datasets.maze as maze
import lib.datasets.mnist as mnist
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.loggers.loggers as loggers
import lib.loggers.logger_utils as logger_utils
from lib.models.models import UniformRate, UniformVariantRate, GaussianTargetRate
import lib.utils.utils as utils
import numpy as np
from lib.datasets import dataset_utils
from torch.utils.data import Dataset, DataLoader


def show_images(images, n=8, figsize=(10, 1), title='mazes.png'):
    plt.figure(figsize=figsize)
    for i in range(n):
        ax = plt.subplot(1, n, i + 1)
        plt.imshow(images[i].permute(1, 2, 0).numpy().astype("uint8"), cmap="gray")
        plt.axis("off")
    plt.savefig(title, transparent=True)
    plt.show()
    #plt.close()



def plot_samples(samples, im_size=0, axis=False, im_fmt=None):
    """Plot samples."""
    plt.scatter(samples[:, 0], samples[:, 1], marker=".")
    plt.axis("equal")
    if im_size > 0:
        plt.xlim(-im_size, im_size)
        plt.ylim(-im_size, im_size)
    if not axis:
        plt.axis("off")
    plt.show()
    plt.close()


# loss_res = np.load('loss_CatRM104999_hollow10MRRes.npy')
def apply_ema(signal, alpha):
    """
    Anwendung des Exponential Moving Average auf ein Signal.

    :param signal: Das Eingangssignal (Liste oder NumPy-Array).
    :param alpha: Der Glättungsfaktor. Niedrigere Werte glätten mehr, typisch zwischen 0 und 1.
    :return: Das geglättete Signal.
    """
    ema = [signal[0]]  # Startwert für EMA
    for i in range(1, len(signal)):
        ema.append(alpha * signal[i] + (1 - alpha) * ema[i - 1])
    return np.array(ema)


"""
res = loss_res[1000:105000]
sm = apply_ema(res, 0.0001)
plt.plot(sm)
plt.yscale('log')
plt.show()
"""


def plot_squared(n_samples, samples, save_location_png):
    fig = plt.figure(figsize=(9, 9))
    for i in range(n_samples):
        plt.subplot(int(np.sqrt(n_samples)), int(np.sqrt(n_samples)), 1 + i)
        plt.axis("off")
        plt.imshow(np.transpose(samples[i, ...], (1, 2, 0)), cmap="gray")

    saving_plot_path = os.path.join(
        save_location_png,
        "mazes.png",
    )
    # plt.suptitle('Mazes', fontsize=16)
    # plt.subplots_adjust(top=0.85)
    # plt.subplots_adjust(wspace=0.1, hspace=0.1)
    # plt.savefig(saving_plot_path)
    plt.show()
    plt.close()

In [ ]:
from config.sudoku_config.config_sudoku_unet import get_config

cfg = get_config()
device = cfg.device
device = "cuda"
cfg.data.name = 'SudokuDataset'
mnist_dataset = dataset_utils.get_dataset(cfg, device, _)
cfg.data.batch_size = 10
mnist_dl = DataLoader(
    mnist_dataset, cfg.data.batch_size, shuffle=True, num_workers=0
)
# worker_init_fn=worker_init_fn)
#cfg.model.time_base = 3
#cfg.model.time_exp = 100
#cfg.model.rate_sigma = 6.0
#cfg.model.Q_sigma = 512.0

In [ ]:
cfg.model.rate_const = 0.4 # 2.3 log_sqr t=0.5T 0.4763, 0.2619, 0.2619 [0.3511, 0.3245, 0.3245], [0.3444, 0.3278, 0.3278], [0.4763, 0.2619, 0.2619], 0.5557, 0.2221, 0.2221],
cfg.model.t_func = "sqrt_cos"  # sqrt_cos

# model = UniformRate(cfg, 'cuda')
model = UniformVariantRate(cfg, "cuda")
S = 2
# 0.25 [0.7867, 0.1116, 0.1116]
# 0.5 [0.4763, 0.2619, 0.2619
# 1 [0.3389, 0.3305, 0.3305],
ls = np.linspace(0.001, 1, 8)
tensor_list = []
for minibatch in mnist_dl:
    B, C, H, W = minibatch.shape
    D = C * H * W
    minibatch = minibatch.view(B, D)
    ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
    ts = torch.ones((B,), device=device) *1# 0.001
    ts = torch.clamp(ts, max=0.99999)
    # ts = torch.linspace(0.01, 1, B, device=device)
    print("Time points", ts[:9])
    qt0 = model.transition(
        ts
    )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

    # R_t = beta_t * R_b
    rate = model.rate(
        ts
    )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

    qt0_rows_reg = qt0[
        torch.arange(B, device=device).repeat_interleave(
            D
        ),  # repeats every element 0 to B-1 D-times
        minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
        :,
    ]  # (B*D, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()].view(-1, S)
    print("Prob", qt0_rows_reg)  # , qt0)
    # set of (B*D) categorical distributions with probabilities from qt0_rows_reg
    x_t_cat = torch.distributions.categorical.Categorical(qt0_rows_reg)
    x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
        B, D
    )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

    rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]  # (B*D, S)

    rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

    rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

    #  Summe der Werte entlang der Dimension S
    rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
        B, D
    )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
    square_dimcat = torch.distributions.categorical.Categorical(
        rate_vals_square_dimsum
    )

    square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

    rate_new_val_probs = rate_vals_square[
        torch.arange(B, device=device), square_dims, :
    ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
    # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

    # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
    square_newvalcat = torch.distributions.categorical.Categorical(
        rate_new_val_probs
    )

    square_newval_samples = (
        square_newvalcat.sample()
    )  # (B, ) taking values in [0, S)

    x_tilde = x_t.clone()
    x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
    x_tilde = x_tilde.view(B, C, H, W)
    # ensures that positive

    reg_x = x_tilde
    outer_rate_sig = rate[
        torch.arange(B, device=device).repeat_interleave(D * S),
        torch.arange(S, device=device).repeat(B * D),
        x_tilde.long().flatten().repeat_interleave(S),
    ].view(B, D, S)
    rate_vals_reg = rate[
            torch.arange(B, device=device).repeat_interleave(D),
            :,
            reg_x.long().flatten(),
        ].view(B, D, S)
    print("Equal", (outer_rate_sig == rate_vals_reg).all())
    #tensor_list.append(x_tilde)
    #show_images(minibatch.view(B, C, H, W).detach().cpu(), n=B) # * 127.5
    #show_images(x_tilde.detach().cpu() * 127.5, n=B) # * 127.5
    # noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
    # show_images(noise_x.view(B, C, H, W).detach().cpu() * 127.5, n=B) # * 127.5
#tens = torch.stack(tensor_list, 0).to(device)
#tens[0] = minibatch.view(C, H, W)
#print(tens.shape)
#show_images(tens.detach().cpu(), n=len(ls))
    break

In [ ]:
from config.bin_mnist_config.config_tauUnet_binmnist import get_config

cfg = get_config()
device = cfg.device
device = "cuda"
cfg.data.name = 'BinMNIST'
mnist_dataset = dataset_utils.get_dataset(cfg, device, cfg.data.location)
cfg.data.batch_size = 10
mnist_dl = DataLoader(
    mnist_dataset, cfg.data.batch_size, shuffle=True, num_workers=0
)
# worker_init_fn=worker_init_fn)
#cfg.model.time_base = 3
#cfg.model.time_exp = 100
#cfg.model.rate_sigma = 6.0
#cfg.model.Q_sigma = 512.0

In [ ]:
cfg.model.rate_const = 1.85 # 2.3 log_sqr t=0.5T 0.4763, 0.2619, 0.2619 [0.3511, 0.3245, 0.3245], [0.3444, 0.3278, 0.3278], [0.4763, 0.2619, 0.2619], 0.5557, 0.2221, 0.2221],
cfg.model.t_func = "sqrt_cos"  # sqrt_cos

# model = UniformRate(cfg, 'cuda')
model = UniformVariantRate(cfg, "cuda")
S = 2
# 0.25 [0.7867, 0.1116, 0.1116]
# 0.5 [0.4763, 0.2619, 0.2619
# 1 [0.3389, 0.3305, 0.3305],
ls = np.linspace(0.001, 1, 8)
tensor_list = []
for minibatch in mnist_dl:
    B, C, H, W = minibatch.shape
    D = C * H * W
    minibatch = minibatch.view(B, D)
    ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
    ts = torch.ones((B,), device=device) * 1
    ts = torch.clamp(ts, max=0.99999)
    # ts = torch.linspace(0.01, 1, B, device=device)
    print("Time points", ts[:9])
    qt0 = model.transition(
        ts
    )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

    # R_t = beta_t * R_b
    rate = model.rate(
        ts
    )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

    qt0_rows_reg = qt0[
        torch.arange(B, device=device).repeat_interleave(
            D
        ),  # repeats every element 0 to B-1 D-times
        minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
        :,
    ]  # (B*D, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()].view(-1, S)
    print("Prob", qt0_rows_reg)  # , qt0)
    # set of (B*D) categorical distributions with probabilities from qt0_rows_reg
    x_t_cat = torch.distributions.categorical.Categorical(qt0_rows_reg)
    x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
        B, D
    )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

    rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]  # (B*D, S)

    rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

    rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

    #  Summe der Werte entlang der Dimension S
    rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
        B, D
    )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
    square_dimcat = torch.distributions.categorical.Categorical(
        rate_vals_square_dimsum
    )

    square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

    rate_new_val_probs = rate_vals_square[
        torch.arange(B, device=device), square_dims, :
    ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
    # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

    # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
    square_newvalcat = torch.distributions.categorical.Categorical(
        rate_new_val_probs
    )

    square_newval_samples = (
        square_newvalcat.sample()
    )  # (B, ) taking values in [0, S)

    x_tilde = x_t.clone()
    x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
    x_tilde = x_tilde.view(B, C, H, W)
    # ensures that positive

    reg_x = x_tilde
    outer_rate_sig = rate[
        torch.arange(B, device=device).repeat_interleave(D * S),
        torch.arange(S, device=device).repeat(B * D),
        x_tilde.long().flatten().repeat_interleave(S),
    ].view(B, D, S)
    rate_vals_reg = rate[
            torch.arange(B, device=device).repeat_interleave(D),
            :,
            reg_x.long().flatten(),
        ].view(B, D, S)
    print("Equal", (outer_rate_sig == rate_vals_reg).all())
    #tensor_list.append(x_tilde)
    #show_images(minibatch.view(B, C, H, W).detach().cpu(), n=B) # * 127.5
    #show_images(x_tilde.detach().cpu() * 127.5, n=B) # * 127.5
    # noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
    # show_images(noise_x.view(B, C, H, W).detach().cpu() * 127.5, n=B) # * 127.5
#tens = torch.stack(tensor_list, 0).to(device)
#tens[0] = minibatch.view(C, H, W)
#print(tens.shape)
#show_images(tens.detach().cpu(), n=len(ls))
    break

In [ ]:
import torch
from torch import nn
from einops.layers.torch import Rearrange
patch_size = 4
image_size_h = image_size_w = 28
num_patches = (image_size_h // patch_size) * (image_size_w // patch_size)
channels = 1
patch_dim = channels * patch_size ** 2
dim = patch_dim
seq = nn.Sequential(
    Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
   #nn.LayerNorm(patch_dim),
    nn.Linear(patch_dim, dim),
    #nn.LayerNorm(dim)
)
x = torch.randn(1, 1, 28, 28)
print(x)
print(seq(x))

In [ ]:
import torch

N = 1  # Anzahl der Matrizen
S = 3 # Anzahl der Zustände

# Erzeuge eine NxSxS-Matrix mit Nullen
state_change_matrix = torch.zeros(S, S)

# Fülle die Hauptdiagonale mit Nullen
state_change_matrix[torch.arange(S), torch.arange(S)] = 0

# Fülle die Werte über der Hauptdiagonalen (nach rechts)
for i in range(1, S):
    state_change_matrix[torch.arange(S - i), torch.arange(i, S)] = i

# Fülle die Werte unter der Hauptdiagonalen (nach links)
for i in range(1, S):
    state_change_matrix[torch.arange(i, S), torch.arange(S - i)] = -i
print(state_change_matrix)
B = 5
# print(torch.tile(state_change_matrix, (B, 1,1)).shape)
torch.save(state_change_matrix, 'state_change_matrix_maze.pth')

In [ ]:
cfg.model.rate_const = 0.012 # 2.3 log_sqr t=0.5T 0.4763, 0.2619, 0.2619 [0.3511, 0.3245, 0.3245], [0.3444, 0.3278, 0.3278], [0.4763, 0.2619, 0.2619], 0.5557, 0.2221, 0.2221],
cfg.model.t_func = "sqrt_cos"  # sqrt_cos

# model = UniformRate(cfg, 'cuda')
model = UniformVariantRate(cfg, "cuda")
S = 256
# 0.25 [0.7867, 0.1116, 0.1116]
# 0.5 [0.4763, 0.2619, 0.2619
# 1 [0.3389, 0.3305, 0.3305],
ls = np.linspace(0.001, 1, 8)
tensor_list = []
for minibatch in mnist_dl:
    B, C, H, W = minibatch.shape
    D = C * H * W
    for i in ls:
        minibatch = minibatch.view(B, D)
        # ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
        ts = torch.ones((B,), device=device) * i
        ts = torch.clamp(ts, max=0.99999)
        # ts = torch.linspace(0.01, 1, B, device=device)
        print("Time points", ts[:9])
        qt0 = model.transition(
            ts
        )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

        # R_t = beta_t * R_b
        rate = model.rate(
            ts
        )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

        qt0_rows_reg = qt0[
            torch.arange(B, device=device).repeat_interleave(
                D
            ),  # repeats every element 0 to B-1 D-times
            minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
            :,
        ]  # (B*D, S)
        b = utils.expand_dims(
            torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
        )
        qt0 = qt0[b, minibatch.long()].view(-1, S)
        print(qt0_rows_reg)  # , qt0)
        # set of (B*D) categorical distributions with probabilities from qt0_rows_reg
        x_t_cat = torch.distributions.categorical.Categorical(qt0_rows_reg)
        x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
            B, D
        )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

        rate_vals_square = rate[
            torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
        ]  # (B*D, S)

        rate_vals_square[
            torch.arange(B * D, device=device), x_t.long().flatten()
        ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

        rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

        #  Summe der Werte entlang der Dimension S
        rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
            B, D
        )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
        square_dimcat = torch.distributions.categorical.Categorical(
            rate_vals_square_dimsum
        )

        square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

        rate_new_val_probs = rate_vals_square[
            torch.arange(B, device=device), square_dims, :
        ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
        # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

        # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
        square_newvalcat = torch.distributions.categorical.Categorical(
            rate_new_val_probs
        )

        square_newval_samples = (
            square_newvalcat.sample()
        )  # (B, ) taking values in [0, S)

        x_tilde = x_t.clone()
        x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
        # x_tilde = x_tilde.view(B, C, H, W)
        x_tilde = x_tilde.view(C, H, W)
        tensor_list.append(x_tilde)
        # show_images(minibatch.view(B, C, H, W).detach().cpu()* 127.5, n=B) # * 127.5
        # show_images(x_tilde.detach().cpu() * 127.5, n=B) # * 127.5
        # noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
        # show_images(noise_x.view(B, C, H, W).detach().cpu() * 127.5, n=B) # * 127.5
    tens = torch.stack(tensor_list, 0).to(device)
    tens[0] = minibatch.view(C, H, W)
    print(tens.shape)
    show_images(tens.detach().cpu(), n=len(ls))
    break

In [ ]:
def show_images2(images, n=8, figsize=(3, 3), title='mazes.png'):
    plt.figure(figsize=figsize)
    plt.imshow(images.permute(1, 2, 0).numpy().astype("uint64"), cmap="gray")
    plt.axis("off")
    plt.savefig(title)
    plt.close()
show_images2(tens[5].view(C, H, W).cpu(), n=1, title='middle_2.png')

In [ ]:
def show_images2(images, n=8, figsize=(3, 3), title='mazes.png'):
    plt.figure(figsize=figsize)
    plt.imshow(images.permute(1, 2, 0).numpy().astype("uint64"), cmap="gray")
    plt.axis("off")
    plt.savefig(title)
    plt.close()
show_images2(tens[0].view(C, H, W).detach().cpu(), n=1, title='true.png')
show_images2(tens[3].view(C, H, W).cpu(), n=1, title='middle.png')
show_images2(tens[-1].view(C, H, W).detach().cpu(), n=1, title='noisy.png')

In [ ]:
cfg.model.rate_const = 0.01 # 2.3 log_sqr t=0.5T 0.4763, 0.2619, 0.2619 [0.3511, 0.3245, 0.3245], [0.3444, 0.3278, 0.3278], [0.4763, 0.2619, 0.2619], 0.5557, 0.2221, 0.2221],
cfg.model.t_func = "sqrt_cos"  # sqrt_cos

# model = UniformRate(cfg, 'cuda')
model_2 = UniformRate(cfg, "cuda")
S = 256
# 0.25 [0.7867, 0.1116, 0.1116]
# 0.5 [0.4763, 0.2619, 0.2619
# 1 [0.3389, 0.3305, 0.3305],
ls = np.linspace(0.001, 1, 8)
tensor_list = []

for i in ls:
    minibatch = minibatch.view(B, D)
    # ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
    ts = torch.ones((B,), device=device) * i
    ts = torch.clamp(ts, max=0.99999)
    # ts = torch.linspace(0.01, 1, B, device=device)
    print("Time points", ts[:9])
    qt0 = model_2.transition(
        ts
    )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

    # R_t = beta_t * R_b
    rate = model_2.rate(
        ts
    )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

    qt0_rows_reg = qt0[
        torch.arange(B, device=device).repeat_interleave(
            D
        ),  # repeats every element 0 to B-1 D-times
        minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
        :,
    ]  # (B*D, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()].view(-1, S)
    print(qt0_rows_reg)  # , qt0)
    # set of (B*D) categorical distributions with probabilities from qt0_rows_reg
    x_t_cat = torch.distributions.categorical.Categorical(qt0_rows_reg)
    x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
        B, D
    )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

    rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]  # (B*D, S)

    rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

    rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

    #  Summe der Werte entlang der Dimension S
    rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
        B, D
    )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
    square_dimcat = torch.distributions.categorical.Categorical(
        rate_vals_square_dimsum
    )

    square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

    rate_new_val_probs = rate_vals_square[
        torch.arange(B, device=device), square_dims, :
    ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
    # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

    # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
    square_newvalcat = torch.distributions.categorical.Categorical(
        rate_new_val_probs
    )

    square_newval_samples = (
        square_newvalcat.sample()
    )  # (B, ) taking values in [0, S)

    x_tilde = x_t.clone()
    x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
    # x_tilde = x_tilde.view(B, C, H, W)
    x_tilde = x_tilde.view(C, H, W)
    tensor_list.append(x_tilde)
    # show_images(minibatch.view(B, C, H, W).detach().cpu()* 127.5, n=B) # * 127.5
    # show_images(x_tilde.detach().cpu() * 127.5, n=B) # * 127.5
    # noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
    # show_images(noise_x.view(B, C, H, W).detach().cpu() * 127.5, n=B) # * 127.5
tens = torch.stack(tensor_list, 0).to(device)
tens[0] = minibatch.view(C, H, W)
show_images(tens.detach().cpu(), n=len(ls))


In [ ]:
show_images(tens[0].detach().cpu(), n=1, title='true.png')
show_images(tens[3].detach().cpu(), n=1, title='middle.png')
show_images(tens[-1].detach().cpu(), n=1, title='noisy.png')

In [ ]:
from config.maze_config.config_tauUnet_maze import get_config

cfg = get_config()
model = model_utils.create_model(cfg, cfg.device)
print("Number of Parameters: ", sum([p.numel() for p in model.parameters()]))

In [ ]:
from lib.networks.ddsm_networks import ProteinScoreNet
from config.maze_config.config_hollow_maze import get_config

cfg = get_config()
cfg.device = "cuda"


D = cfg.model.concat_dim = 1024
S = cfg.data.S = 5
B = cfg.data.batch_size
net = ProteinScoreNet(cfg).to(cfg.device)
ts = torch.rand((B,), device=cfg.device) * (1.0 - 0.01) + 0.01
x = torch.randint(low=0, high=S, size=(B, D), device="cuda")
x_out = net(x, ts)
print(x_out.shape)

In [ ]:
from config.mnist_config.config_hollow_mnist import get_config

cfg = get_config()
device = cfg.device
device = "cuda"
#cfg.data.name = 'DiscreteCIFAR10'
mnist_dataset = dataset_utils.get_dataset(cfg, device, cfg.data.location)
cfg.data.batch_size = 16
mnist_dl = DataLoader(
    mnist_dataset, cfg.data.batch_size, shuffle=cfg.data.shuffle, num_workers=0
)
# worker_init_fn=worker_init_fn)
#cfg.model.time_base = 3
#cfg.model.time_exp = 100
#cfg.model.rate_sigma = 6.0
#cfg.model.Q_sigma = 512.0

#model = GaussianTargetRate(cfg, device)
n_samples = cfg.data.batch_size
for samples in mnist_dl:
    print(samples.shape)
    samples = samples.cpu().numpy()
    samples = samples.reshape(n_samples, 1, cfg.data.image_size, cfg.data.image_size)
    #saving_train_path = os.path.join(cfg.saving.sample_plot_path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
    fig = plt.figure(figsize=(9, 9)) 
    for i in range(n_samples):
        plt.subplot(int(np.sqrt(n_samples)), int(np.sqrt(n_samples)), 1 + i)
        plt.axis("off")
        plt.imshow(np.transpose(samples[i, ...], (1,2,0)), cmap="gray")
 
 
    plt.savefig('mnist_data.png', bbox_inches='tight', transparent=True)
    plt.show()

    break

In [ ]:
from config.synthetic_config.config_hollow_synthetic import get_config

cfg = get_config()
location = "lib/datasets/Synthetic/data_2spirals.npy"
device = cfg.device
device = "cuda"
synthetic_dataset = dataset_utils.get_dataset(cfg, device, location)
cfg.data.batch_size = 1500
synthetic_dl = DataLoader(
    synthetic_dataset, cfg.data.batch_size, shuffle=cfg.data.shuffle, num_workers=0
)



In [ ]:
cfg.model.rate_const = 50000000 # 0.00009
cfg.model.t_func = "sqrt_cos"
#model = UniformRate(cfg, device)
model = UniformVariantRate(cfg, device)
# model = GaussianTargetRate(cfg, device)
S = 2
bm, inv_bm = synthetic.get_binmap(cfg.model.concat_dim, cfg.data.binmode)
ls = np.linspace(0.001, 1, 10)
for minibatch in synthetic_dl:
#for i in ls:
    # print(minibatch.device)
    B, D = minibatch.shape
    minibatch = minibatch.view(B, D).to("cuda")
    ts = torch.ones((B,), device=device) * 0.0000000000000001 #0.00001


    qt0 = model.transition(
        ts
    )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

    # R_t = beta_t * R_b
    rate = model.rate(
        ts
    )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

    qt0_rows_reg = qt0[
        torch.arange(B, device=device).repeat_interleave(
            D
        ),  # repeats every element 0 to B-1 D-times
        minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
        :,
    ]  # (B*D, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()]
    print(qt0)
    log_qt0 = torch.where(qt0_rows_reg <= 0.0, -1e9, torch.log(qt0_rows_reg))
    #print(log_qt0)
    x_t_cat = torch.distributions.categorical.Categorical(logits=log_qt0)
    x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
        B, D
    )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

    rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]  # (B*D, S)

    rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

    rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

    #  Summe der Werte entlang der Dimension S
    rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
        B, D
    )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
    
    square_dimcat = torch.distributions.categorical.Categorical(
        rate_vals_square_dimsum
    )

    square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

    rate_new_val_probs = rate_vals_square[
        torch.arange(B, device=device), square_dims, :
    ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
    # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

    # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
    square_newvalcat = torch.distributions.categorical.Categorical(
        rate_new_val_probs
    )
    print(rate_new_val_probs)

    square_newval_samples = (
        square_newvalcat.sample()
    )  # (B, ) taking values in [0, S)

    x_tilde = x_t.clone()
    x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
    x_tilde = synthetic.bin2float(
        x_tilde.detach().cpu().numpy().astype(np.int32),
        inv_bm,
        cfg.model.concat_dim,
        cfg.data.int_scale,
    )

    minibatch = synthetic.bin2float(minibatch.detach().cpu().numpy().astype(np.int32), inv_bm, cfg.model.concat_dim, cfg.data.int_scale)
    #x_tilde = synthetic.bin2float(x_tilde.detach().cpu().numpy().astype(np.int32), inv_bm, cfg.model.concat_dim, cfg.data.int_scale)
    #plot_samples(minibatch) # * 127.5
    plot_samples(x_tilde) # * 127.5
    #noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)

    # noise_x = synthetic.bin2float(noise_x.detach().cpu().numpy().astype(np.int32), inv_bm, cfg.model.concat_dim, cfg.data.int_scale)
    # plot_samples(noise_x) # * 127.5
    break

In [ ]:
cfg.model.rate_const = 0.1  # 0.00009
cfg.model.t_func = "log"

cfg.model.time_base = 3
cfg.model.time_exp = 100
cfg.model.rate_sigma = 6.0
cfg.model.Q_sigma = 512.0
# model = UniformRate(cfg, device)
model = UniformVariantRate(cfg, device)
# model = GaussianTargetRate(cfg, device)
S = 256


for minibatch in mnist_dl:
    B, C, H, W = minibatch.shape
    print(minibatch.device)
    D = C * H * W
    minibatch = minibatch.view(B, D).to("cpu")
    ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
    ts = torch.ones((B,), device=device) * 1
    print("Time points", ts[:9])

    qt0 = model.transition(
        ts
    )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

    # R_t = beta_t * R_b
    rate = model.rate(
        ts
    )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

    qt0_rows_reg = qt0[
        torch.arange(B, device=device).repeat_interleave(
            D
        ),  # repeats every element 0 to B-1 D-times
        minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
        :,
    ]  # (B*D, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()]
    print(qt0_rows_reg, qt0_rows_reg.shape)
    # set of (B*D) categorical distributions with probabilities from qt0_rows_reg
    x_t_cat = torch.distributions.categorical.Categorical(qt0_rows_reg)
    x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
        B, D
    )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

    rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]  # (B*D, S)

    rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

    rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

    #  Summe der Werte entlang der Dimension S
    rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
        B, D
    )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
    square_dimcat = torch.distributions.categorical.Categorical(rate_vals_square_dimsum)

    square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

    rate_new_val_probs = rate_vals_square[
        torch.arange(B, device=device), square_dims, :
    ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
    # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

    # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
    square_newvalcat = torch.distributions.categorical.Categorical(rate_new_val_probs)

    square_newval_samples = square_newvalcat.sample()  # (B, ) taking values in [0, S)

    x_tilde = x_t.clone()
    x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
    x_tilde = x_tilde.view(B, C, H, W)

    show_images(minibatch.view(B, C, H, W).detach().cpu(), n=9)  # * 127.5
    show_images(x_tilde.detach().cpu(), n=9)  # * 127.5
    noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
    show_images(noise_x.view(B, C, H, W).detach().cpu(), n=9)  # * 127.5
    break

In [ ]:
from config.maze_config.config_hollow_maze import get_config

cfg = get_config()
device = cfg.device
device = "cuda"
maze_dataset = dataset_utils.get_dataset(cfg, device)
cfg.data.batch_size = 16
maze_dl = DataLoader(
    maze_dataset, cfg.data.batch_size, shuffle=cfg.data.shuffle, num_workers=0
)
# worker_init_fn=worker_init_fn)

In [ ]:
cfg.model.rate_const = 1.7  # 2.3 log_sqr t=0.5T 0.4763, 0.2619, 0.2619 [0.3511, 0.3245, 0.3245], [0.3444, 0.3278, 0.3278], [0.4763, 0.2619, 0.2619], 0.5557, 0.2221, 0.2221],
cfg.model.t_func = "sqrt_cos"  # sqrt_cos
cfg.model.time_base = 1
cfg.model.time_exp = 250
#model = UniformRate(cfg, 'cuda')
model = UniformVariantRate(cfg, "cuda")
S = 3
# 0.25 [0.7867, 0.1116, 0.1116]
# 0.5 [0.4763, 0.2619, 0.2619
# 1 [0.3389, 0.3305, 0.3305],
ls = np.linspace(0.001, 1, 10)
tensor_list = []
for minibatch in maze_dl:
    B, C, H, W = minibatch.shape
    D = C * H * W
    minibatch = minibatch.view(B, D)
    # ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
    ts = torch.ones((B,), device=device) * 0.9999 #0.99#0.0001
    # ts = torch.linspace(0.01, 1, B, device=device)

    qt0 = model.transition(
        ts
    )  # (B, S, S) # transition q_{t | s=0} eq.15 => here randomness because of ts => for every ts another q_{t|0}

    # R_t = beta_t * R_b
    rate = model.rate(
        ts
    )  # (B, S, S) # no proability in here (diagonal = - sum of rows)

    qt0_rows_reg = qt0[
        torch.arange(B, device=device).repeat_interleave(
            D
        ),  # repeats every element 0 to B-1 D-times
        minibatch.flatten().long(),  # minibatch.flatten() => (B, D) => (B*D) (1D-Tensor)
        :,
    ]  # (B*D, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()].view(-1, S)
    print(qt0_rows_reg)  # , qt0)
    # set of (B*D) categorical distributions with probabilities from qt0_rows_reg
    log_qt0 = torch.where(qt0_rows_reg <= 0.0, -1e9, torch.log(qt0_rows_reg))
    x_t_cat = torch.distributions.categorical.Categorical(logits=log_qt0)
    x_t = x_t_cat.sample().view(  # sampling B * D times => from every row of qt0_rows_reg once => then transform it to shape B, D
        B, D
    )  # (B*D,) mit view => (B, D) Bsp: x_t = (0, 1, 2, 4, 3) (for B =1 )

    rate_vals_square = rate[
        torch.arange(B, device=device).repeat_interleave(D), x_t.long().flatten(), :
    ]  # (B*D, S)

    rate_vals_square[
        torch.arange(B * D, device=device), x_t.long().flatten()
    ] = 0.0  # - values = 0 => in rate_vals_square[0, 1] = 0

    rate_vals_square = rate_vals_square.view(B, D, S)  # (B*D, S) => (B, D, S)

    #  Summe der Werte entlang der Dimension S
    rate_vals_square_dimsum = torch.sum(rate_vals_square, dim=2).view(
        B, D
    )  # B, D with every entry = S-1? => for entries of x_t same prob to transition?
    square_dimcat = torch.distributions.categorical.Categorical(
        rate_vals_square_dimsum
    )

    square_dims = square_dimcat.sample()  # (B,) taking values in [0, D)

    rate_new_val_probs = rate_vals_square[
        torch.arange(B, device=device), square_dims, :
    ]  # (B, S) => every row has only one entry = 0, everywhere else 1; chooses the row square_dim of rate_vals_square
    # => now rate_new_val_probs: (B, S) with every row (1, 1, 0)

    # samples from rate_new_val_probs and chooses state to transition to => more likely where entry is 1 instead of 0?
    square_newvalcat = torch.distributions.categorical.Categorical(
        rate_new_val_probs
    )

    square_newval_samples = (
        square_newvalcat.sample()
    )  # (B, ) taking values in [0, S)

    x_tilde = x_t.clone()
    x_tilde[torch.arange(B, device=device), square_dims] = square_newval_samples
    # x_tilde = x_tilde.view(B, C, H, W)
    x_tilde = x_tilde.view(B, C, H, W)
    tensor_list.append(x_tilde)
    np.save('mazes.npy', minibatch.cpu().numpy())
    #show_images(minibatch.view(B, C, H, W).detach().cpu()* 127.5, n=B) # * 127.5
    #show_images(x_tilde.detach().cpu() * 127.5, n=B, title='mazes_noisy2.png') # * 127.5
    # noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
    # show_images(noise_x.view(B, C, H, W).detach().cpu() * 127.5, n=B) # * 127.5

    break

In [ ]:
samples = minibatch.cpu().numpy().reshape(cfg.data.batch_size , 1, cfg.data.image_size, cfg.data.image_size)
#saving_train_path = os.path.join(cfg.saving.sample_plot_path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
#fig = plt.figure(figsize=(9, 9)) 
for i in range(cfg.data.batch_size):
    plt.subplot(int(np.sqrt(cfg.data.batch_size )), int(np.sqrt(cfg.data.batch_size)), 1 + i)
    plt.axis("off")
    plt.imshow(np.transpose(samples[i, ...], (1,2,0)), cmap="gray")


#plt.savefig(saving_train_path)
    
plt.show()
plt.close()

In [ ]:
for minibatch in maze_dl:
    B, C, H, W = minibatch.shape
    show_images(minibatch.view(B, C, H, W).detach().cpu() * 127.5, n=8)
    break

In [ ]:
import lib.utils.utils as utils

cfg.model.rate_const = 0.5
cfg.model.t_func = "log"
cfg.model.time_base = 5
cfg.model.time_exp = 5
model = UniformRate(cfg, "cuda")
model = UniformVariantRate(cfg, "cuda")
device = "cpu"
S = 3
min_time = 0.01


for minibatch in mnist_dataset:
    if len(minibatch.shape) == 4:
        B, C, H, W = minibatch.shape
        minibatch = minibatch.view(B, C * H * W)
    # hollow xt, t, l_all, l_xt geht rein
    B = minibatch.shape[0]
    ts = torch.rand((B,), device=device) * (1.0 - min_time) + min_time
    ts = torch.ones((B,)) * 1
    print(ts[:9])
    #

    qt0 = model.transition(ts)  # (B, S, S)

    # rate = model.rate(ts)  # (B, S, S)

    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )
    qt0 = qt0[b, minibatch.long()]

    # log loss
    log_qt0 = torch.where(qt0 <= 0.0, -1e9, torch.log(qt0))
    x_tilde = torch.distributions.categorical.Categorical(
        logits=log_qt0
    ).sample()  # bis hierhin <1 sek

    x_tilde = x_tilde.view(B, C, H, W)
    print(torch.mean(x_tilde[1, :, :, :].float()))
    # print(x_tilde[0,0, :, :].std())
    show_images(minibatch.view(B, C, H, W) * 127.5, n=9)
    show_images(x_tilde * 127.5, n=9)
    noise_x = torch.randint(low=0, high=S, size=(B, D), device=device)
    show_images(noise_x.view(B, C, H, W) * 127.5, n=9)
    break

In [ ]:
# int log(t**2 + 1)
t = np.linspace(0.01, 1, 1000)
f = 2 * t / (t**2 + 1)
f_int = np.log(t**2 + 1)
f_cos = np.sin(t) / np.sqrt(np.cos(t))
a = 5
b = 5
f_log = a * np.log(b) * b**t
plt.plot(f, label="log sqr")
plt.plot(f_int, label="int log sqr")
plt.plot(f_cos, label="cos")
plt.legend()
plt.show()